In [35]:
from sympy import *
import numpy as np
from IPython.display import display, Math, Latex

In [36]:
# Defining symbols
H, B, HB, H2B, H3B = symbols(r'[H^{+}], [B^{3-}], [HB^{2-}], [H_{2}B^-], [H_{3}B]',
                                           positive=True, real=True)
K_1, K_2, K_3, K_1_pr, K_2_pr, K_3_pr = symbols(r'K_1, K_2, K_3, K_1^{\prime}, K_2^{\prime}, K_3^{\prime}',
                                           positive=True, real=True)
kf_1, kb_1, kf_2, kb_2 = symbols(r'k_f^1, k_b^1, k_f^2, k_b^2',
                                 positive=True, real=True)
y_H, y_H3B, y_H2B, y_HB, y_B = symbols(r'y_{H^+}, y_{H_{3}B}, y_{H_{2}B^-}, y_{HB^{2-}}, y_{B^{3-}}',
                                   positive=True, real=True)
I, pH, C_t, C_sup = symbols(r'I, pH, C_{total}, C_{sup}',
                    positive=True, real=True)

In [79]:
class AcidBaseEquilibrium:
    def __init__(self, C_tot, pH0, pK_values, I_add = 0):
        self.C_tot = C_tot
        self.pH0 = pH0
        self.pK_values = pK_values
        self.I_add = I_add
        self.num_pK = len(pK_values)
        self.y_H_expr = 10**(-0.5115*1*((sqrt(I) / (1+sqrt(I))) - 0.3*I))
        self.y_H3B_expr = 10**(-0.5115*0*((sqrt(I) / (1+sqrt(I))) - 0.3*I))
        self.y_H2B_expr = 10**(-0.5115*1*((sqrt(I) / (1+sqrt(I))) - 0.3*I))
        self.y_HB_expr = 10**(-0.5115*4*((sqrt(I) / (1+sqrt(I))) - 0.3*I))
        self.y_B_expr = 10**(-0.5115*9*((sqrt(I) / (1+sqrt(I))) - 0.3*I))
        
        #self.K_1_expr = (H * H2B / H3B) * (y_H * y_H2B / y_H3B)
        #self.K_2_expr = (H * HB / H2B) * (y_H * y_HB / y_H2B)
        #self.K_3_expr = (H * B / HB) * (y_H * y_B / y_HB)
        self.real_params = {C_t: C_tot,
                            pH: pH0}
        
        if self.num_pK == 1:
            self.init_monoacid()
        elif self.num_pK == 2:
            self.init_diacid()
        elif self.num_pK == 3:
            self.init_triacid()
        else:
            raise ValueError("Supported only from 1 to 3 pKa values")
            
    def init_monoacid(self):
        global H2B, H3B, y_H2B, y_H3B
        H2B, H3B = symbols(r'[B^{-}], [HB]', positive=True, real=True)
        y_H2B, y_H3B = symbols(r'y_{B^{-}}, y_{HB}', positive=True, real=True)
        display(Latex(r"Expression for the ionic force according to $I=\frac{1}{2} \sum_{i}{c_i z_i^2}$ is:"))
        I_expr = 0.5*(H3B*0 + H2B*1 + (0*H3B + H2B)*1) + self.I_add
        display(Eq(I, I_expr))
        display(Latex(r"The Davies activity coefficients for the species according to $\log \gamma_i=-A z_i^2\left(\frac{\sqrt{I}}{1+\sqrt{I}}-0.3 I\right)$ are:"))
        display(Eq(y_H, self.y_H_expr))
        display(Eq(y_H3B, self.y_H3B_expr))
        display(Eq(y_H2B, self.y_H2B_expr))
        K_1_expr = (H * H2B / H3B) * (y_H * y_H2B / y_H3B)
        Eq1 = Eq(K_1, K_1_expr)
        print("We will solve the following system of equations:")
        display(Eq(H2B+H3B, C_t), Eq1)
        init_subs = {y_H3B: self.y_H3B_expr,
                     y_H2B: self.y_H2B_expr,
                     H: 10**(-pH)/y_H,
                     y_H: self.y_H_expr}
        
        Eq1_complete = Eq1.subs(init_subs)
        self.real_params.update({K_1: 10**(-self.pK_values[0])})
        self.dummy_system =[eq.subs(self.real_params).subs(I, 0) for eq in [Eq(H2B+H3B, C_t), Eq1_complete]]
        self.system =[eq.subs(self.real_params).subs(I, I_expr) for eq in [Eq(H2B+H3B, C_t), Eq1_complete]]
        #display(self.system)
        
    def init_diacid(self):
        global HB, H2B, H3B, y_HB, y_H2B, y_H3B
        HB, H2B, H3B = symbols(r'[B^{2-}], [HB^{-}], [H_{2}B]', positive=True, real=True)
        y_HB, y_H2B, y_H3B = symbols(r'y_{B^{2-}}, y_{HB^{-}}, y_{H_{2}B}', positive=True, real=True)
        display(Latex(r"Expression for the ionic force according to $I=\frac{1}{2} \sum_{i}{c_i z_i^2}$ is:"))
        I_expr = 0.5*(H3B*0 + H2B*1 + HB*4 + (0*H3B + H2B + 2*HB)*1) + self.I_add
        display(Eq(I, I_expr))
        display(Latex(r"The Davies activity coefficients for the species according to $\log \gamma_i=-A z_i^2\left(\frac{\sqrt{I}}{1+\sqrt{I}}-0.3 I\right)$ are:"))
        display(Eq(y_H, self.y_H_expr))
        display(Eq(y_H3B, self.y_H3B_expr))
        display(Eq(y_H2B, self.y_H2B_expr))
        display(Eq(y_HB, self.y_HB_expr))
        K_1_expr = (H * H2B / H3B) * (y_H * y_H2B / y_H3B)
        K_2_expr = (H * HB / H2B) * (y_H * y_HB / y_H2B)
        Eq1 = Eq(K_1, K_1_expr)
        Eq2 = Eq(K_2, K_2_expr)
        print("We will solve the following system of equations:")
        display(Eq(H2B+H3B+HB, C_t), Eq1, Eq2)
        init_subs = {y_H3B: self.y_H3B_expr,
                     y_H2B: self.y_H2B_expr,
                     y_HB: self.y_HB_expr,
                     H: 10**(-pH)/y_H,
                     y_H: self.y_H_expr}
        Eq1_complete = Eq1.subs(init_subs)
        Eq2_complete = Eq2.subs(init_subs)
        self.real_params.update({K_1: 10**(-self.pK_values[0]),
                                K_2: 10**(-self.pK_values[1])})
        self.dummy_system =[eq.subs(self.real_params).subs(I, 0) for eq in [Eq(H2B+H3B+HB, C_t), Eq1_complete, Eq2_complete]]
        self.system =[eq.subs(self.real_params).subs(I, I_expr) for eq in [Eq(H2B+H3B+HB, C_t), Eq1_complete, Eq2_complete]]
        #display(self.system)

    def init_triacid(self):
        global B, HB, H2B, H3B, y_B, y_HB, y_H2B, y_H3B
        B, HB, H2B, H3B = symbols(r'[B^{3-}], [HB^{2-}], [H_{2}B^{-}], [H_{3}B]', positive=True, real=True)
        y_B, y_HB, y_H2B, y_H3B = symbols(r'y_{B^{3-}}, y_{HB^{2-}}, y_{H_{2}B^{-}}, y_{H_{3}B}', positive=True, real=True)
        display(Latex(r"Expression for the ionic force according to $I=\frac{1}{2} \sum_{i}{c_i z_i^2}$ is:"))
        I_expr = 0.5*(H3B*0 + H2B*1 + HB*4 + B*9 + (0*H3B + H2B + 2*HB + 3*B)*1) + self.I_add
        display(Eq(I, I_expr))
        display(Latex(r"The Davies activity coefficients for the species according to $\log \gamma_i=-A z_i^2\left(\frac{\sqrt{I}}{1+\sqrt{I}}-0.3 I\right)$ are:"))
        display(Eq(y_H, self.y_H_expr))
        display(Eq(y_H3B, self.y_H3B_expr))
        display(Eq(y_H2B, self.y_H2B_expr))
        display(Eq(y_HB, self.y_HB_expr))
        display(Eq(y_B, self.y_B_expr))
        K_1_expr = (H * H2B / H3B) * (y_H * y_H2B / y_H3B)
        K_2_expr = (H * HB / H2B) * (y_H * y_HB / y_H2B)
        K_3_expr = (H * B / HB) * (y_H * y_B / y_HB)
        Eq1 = Eq(K_1, K_1_expr)
        Eq2 = Eq(K_2, K_2_expr)
        Eq3 = Eq(K_3, K_3_expr)
        print("We will solve the following system of equations:")
        display(Eq(H2B+H3B+HB+B, C_t), Eq1, Eq2, Eq3)
        init_subs = {y_H3B: self.y_H3B_expr,
                     y_H2B: self.y_H2B_expr,
                     y_HB: self.y_HB_expr,
                     y_B: self.y_B_expr,
                     H: 10**(-pH)/y_H,
                     y_H: self.y_H_expr}
        Eq1_complete = Eq1.subs(init_subs)
        Eq2_complete = Eq2.subs(init_subs)
        Eq3_complete = Eq3.subs(init_subs)
        self.real_params.update({K_1: 10**(-self.pK_values[0]),
                                K_2: 10**(-self.pK_values[1]),
                                K_3: 10**(-self.pK_values[2])})
        self.dummy_system =[eq.subs(self.real_params).subs(I, 0) for eq in [Eq(H2B+H3B+HB+B, C_t), Eq1_complete, Eq2_complete, Eq3_complete]]
        self.system =[eq.subs(self.real_params).subs(I, I_expr) for eq in [Eq(H2B+H3B+HB+B, C_t), Eq1_complete, Eq2_complete, Eq3_complete]]
        #display(self.system)
        
    def solve_dummy(self):
        if not hasattr(self, 'dummy_solution'):
            self.dummy_solution = solve(self.dummy_system, dict=True)[0]
        #display(self.dummy_solution)
        
    def solve(self):
        self.solve_dummy()
        if self.num_pK == 1:
            solution = nsolve(self.system, (H3B, H2B), (self.dummy_solution[H3B], self.dummy_solution[H2B]), dict=True, prec=20)[0]
        elif self.num_pK == 2:
            solution = nsolve(self.system, (H3B, H2B, HB), (self.dummy_solution[H3B], self.dummy_solution[H2B], self.dummy_solution[HB]), dict=True, prec=20)[0]
        elif self.num_pK == 3:
            solution = nsolve(self.system, (H3B, H2B, HB, B), (self.dummy_solution[H3B], self.dummy_solution[H2B], self.dummy_solution[HB], self.dummy_solution[B]), dict=True, prec=20)[0]
        for k,v in solution.items():
            display(Latex(fr"The concentration of ${latex(k)}$ is: {v} M"))

In [84]:
# defining the buffer system
buffer = AcidBaseEquilibrium(C_tot = 0.1,                # Total buffer concentration in M
                             pH0 = 5.5,                  # Initial pH 
                             pK_values = [3,5,6],      # List of 1-3 pKa in the increasing order
                             I_add = 0)                  # Additional ionic strength from supporting electrolyte, if present

<IPython.core.display.Latex object>

Eq(I, 6.0*[B^{3-}] + 3.0*[HB^{2-}] + 1.0*[H_{2}B^{-}])

<IPython.core.display.Latex object>

Eq(y_{H^+}, 10**(-0.5115*sqrt(I)/(sqrt(I) + 1) + 0.15345*I))

Eq(y_{H_{3}B}, 1)

...

We will solve the following system of equations:


Eq([B^{3-}] + [HB^{2-}] + [H_{2}B^{-}] + [H_{3}B], C_{total})

Eq(K_1, [H^{+}]*[H_{2}B^{-}]*y_{H^+}*y_{H_{2}B^{-}}/([H_{3}B]*y_{H_{3}B}))

Eq(K_2, [HB^{2-}]*[H^{+}]*y_{HB^{2-}}*y_{H^+}/([H_{2}B^{-}]*y_{H_{2}B^{-}}))

Eq(K_3, [B^{3-}]*[H^{+}]*y_{B^{3-}}*y_{H^+}/([HB^{2-}]*y_{HB^{2-}}))

In [85]:
# Calculating the initial concentrations:
buffer.solve()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>